In [2]:
from insolAPI.WebAPI import API
import simplejson as json
import pandas as pd
import pendulum as pdl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from tqdm import tqdm
import sys

In [3]:
time_args = dict(
    start=pdl.yesterday().subtract(months=1).to_datetime_string(),
    stop=(pdl.yesterday()).to_datetime_string(),
    timezone="UTC",
)

def read_json_config():
    """
    Read the config.json file and return the api key
    """
    with open("config.json") as f:
        config_data = json.load(f)

    installations = {}

    # Iterate through locations
    locations = config_data['locations']
    for location in locations:
        # Store details in a dictionary
        location_details = {
            'id': location['id'],
            'name': location['name'],
            'latitude': location['latitude'],
            'longitude': location['longitude'],
            'wind_threshold': location['wind_threshold'],
            'high_temperature_threshold': location['high_temperature_threshold'],
            'has_a_screen': location['has_a_screen'],
        }
        installations[location['name']] = location_details
    return installations, [config_data['api_key'], config_data['api_url']]


installation_path = "C:/Users/Insolight/Desktop/InsolReports/Scripts"
with open(installation_path + "/local.json") as f:
    local_data = json.load(f)
api = API(local_data["API_user"], local_data["API_pwd"], dev_space=False)

list_sensor = [
    "PAR",
    "IRRAD",
    "GII",
    "DNI",
    "DHI",
    "TEMP",
    "HUMI",
    "RAIN",
    "RAIN_TYPE",
    "RAIN_ACCUMULATED",
    "WIND",
    "WIND_DIR",
    "VIRTUAL",
    "LEAF_TEMP"
]

dict_instal = {
    "Etchelecu": 26,
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
}

# for all sensors

In [6]:
sensor_number = 0
logs_joined = {}
dict_merged_df = {}
dict_df_missing_values = {}
df_log_unique_sensor = {}
for instal in dict_instal:
    print(instal)
    sensor_number = 0
    logs_joined[instal] = {}
    dict_merged_df[instal] = {}
    dict_df_missing_values[instal] = {}
    df_log_unique_sensor[instal] = {}
    for sensor_type in tqdm(api.SensorsTypes):
        if str(sensor_type).split(".")[1] in list_sensor:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                logs_joined[instal][list_sensor[sensor_number]] = api.get_sensor_channels_logs_joined(**time_args,sensor_type=sensor_type, install=dict_instal[instal])
                sensor_number += 1

    for sensor in list_sensor:
        if logs_joined[instal][sensor].empty:
            continue
        list_unique_sensors = logs_joined[instal][sensor]['sensor_name'].unique()
        for unique_sensor in list_unique_sensors:
            df_log_unique_sensor[instal][unique_sensor] = logs_joined[instal][sensor].loc[logs_joined[instal][sensor]['sensor_name']==unique_sensor].copy()
            df_log_unique_sensor[instal][unique_sensor].reset_index(inplace=True)
            df_log_unique_sensor[instal][unique_sensor].rename(columns={'index':'time'}, inplace=True)
            df_log_unique_sensor[instal][unique_sensor]['time'] = pd.to_datetime(df_log_unique_sensor[instal][unique_sensor]['time'])
            df_log_unique_sensor[instal][unique_sensor]['time'] = df_log_unique_sensor[instal][unique_sensor]['time'].dt.round('10min')
            df_log_unique_sensor[instal][unique_sensor].drop_duplicates(subset=['time','sensor_name'], inplace=True)
            date_range = pd.date_range(start=df_log_unique_sensor[instal][unique_sensor]['time'].min(), end=df_log_unique_sensor[instal][unique_sensor]['time'].max() + pd.DateOffset(0), freq='10min')
            full_df = pd.DataFrame(date_range, columns=['time'])
            dict_merged_df[instal][unique_sensor] = pd.merge(full_df, df_log_unique_sensor[instal][unique_sensor], how='left', on="time")
            columns = dict_merged_df[instal][unique_sensor].columns
            dict_merged_df[instal][unique_sensor].rename(columns={columns[2]:'value_sensor'}, inplace=True)

            #counting missing values
            old_day = 0
            missing_values = 0
            dict_days = {}
            for index, row in dict_merged_df[instal][unique_sensor].iterrows():
                new_day = row['time'].day
                if new_day == old_day and pd.isnull(row["value_sensor"]):
                    missing_values += 1
                if new_day != old_day:
                    dict_days[datetime(year=row["time"].year, month=row["time"].month, day=row["time"].day) - timedelta(days=1)] = missing_values*10 / 1430 * 100
                    old_day = new_day
                    missing_values = 0
            dict_df_missing_values[instal][unique_sensor] = pd.DataFrame.from_dict(dict_days, orient='index')

    fig = go.Figure()
    for i in dict_df_missing_values[instal]:
        fig.add_trace(go.Scatter(x=dict_df_missing_values[instal][i].index, y=dict_df_missing_values[instal][i][0], name=i))
        fig.update_layout(yaxis_title='% of day disconnected', hovermode='x', title=instal, hoverlabel=dict(bgcolor="white"))
        #add a title

    fig.show()
    #save figure as html in a folder named "results"
    fig.write_html("missing_sensors_recap/" + instal + '_missing_values.html')

Etchelecu


100%|██████████| 25/25 [00:12<00:00,  1.95it/s]


Agroscope Series


100%|██████████| 25/25 [01:02<00:00,  2.49s/it]


Agroscope Berries


100%|██████████| 25/25 [00:33<00:00,  1.33s/it]


Bioschmid


100%|██████████| 25/25 [00:17<00:00,  1.42it/s]


In [28]:
dict_df_per_day = {}
for instal in dict_instal:
    dict_df_per_day[instal] = pd.DataFrame()
    for i in dict_df_missing_values[instal]:
        # print(i)
        dict_df_per_day[instal][i] = dict_df_missing_values[instal][i][0]

# test

In [8]:
import pickle

# Your dictionary with DataFrames
# your_dict = {'df1': df1, 'df2': df2, 'df3': df3}

# Serialize and save the dictionary to a file
with open('your_data.pkl', 'wb') as file:
    pickle.dump(dict_df_missing_values, file)